In [6]:
from binance import Client
import pandas as pd
import ta

In [17]:
%run ./Binance_Keys.ipynb
client = Client(api_key,api_secret)

In [18]:
from time import sleep
from binance.exceptions import BinanceAPIException

In [19]:
def getminutedata(symbol):
    try:
        df = pd.DataFrame(client.get_historical_klines(symbol, '1m', '40m UTC'))
    except BinanceAPIException as e:
        print(e)
        sleep(60)
        df = pd.DataFrame(client.get_historical_klines(symbol, '1m', '40m UTC'))
    df = df.iloc[:,:6]
    df.columns = ['Time','Open','High','Low','Close','Volume']
    df = df.set_index('Time')
    df.index = pd.to_datetime(df.index, unit='ms')
    df = df.astype(float)
    return df

In [ ]:
def tradingstrat(symbol, qty, open_position = False):
    while True:
        df = getminutedata(symbol)
        if not open_position:
            if ta.trend.macd_diff(df.Close).isloc[-1] > 0 \
            and ta.trend.macd_diff(df.close).isloc[-2] < 0:
                order = client.create_order(symbol=symbol,
                                            side='BUY',
                                            type='Market', quantity=qty)
                print(order)
                open_position = True
                buyprice = order(['fills'][0]['price'])
                break
    if open_position:
        while True:
            df= getminutedata(symbol)
            if ta.trend.macd_diff(df.Close).isloc[-1] < 0 \
            and ta.trend.macd_diff(df.close).isloc[-2] > 0:
                order = client.create_order(symbol=symbol,
                                            side='SELL',
                                            type='Market', quantity=qty)
                print(order)
                sellprice = order(['fills'][0]['price'])
                print(f'iprofit = {(sellprice - buyprice)/buyprice}')
                open_position = False
                break 

In [ ]:
while True:
    tradingstrat('BTCUSDT',qty=0.1)